In [ ]:
import os
import pprint


# Set the environment variables
os.environ['API_KEY'] = 'tlk_0PH966V0F2TH882CN6Q650JD0G5D'
os.environ['API_URL'] = 'https://api.twelvelabs.io/v1.2'

In [ ]:
import requests
import glob
from pprint import pprint
import os

API_URL = os.getenv("API_URL")
assert API_URL

API_KEY = os.getenv("API_KEY")
assert API_KEY

INDEXES_URL = f"{API_URL}/indexes"

INDEX_NAME = "Test1" # Use a descriptive name for your index

headers = {
	"x-api-key": API_KEY
}

data = {
  "engines": [
    {
      "engine_name": "marengo2.5",
      "engine_options": ["visual", "conversation", "text_in_video", "logo"]
    },
    {
      "engine_name": "pegasus1",
      "engine_options": ["visual", "conversation"]
    }
  ],
  "index_name": INDEX_NAME,
  "addons": ["thumbnail"] # (Optional) This line enables the logo detection feature.
}

response = requests.post(INDEXES_URL, headers=headers, json=data)
INDEX_ID = response.json().get('_id')
print (f'Status code: {response.status_code}')
pprint (response.json())

Status code: 409
{'code': 'index_name_already_exists',
 'message': 'Index name Test1 already exists. Please use another unique name '
            'and try again.'}


In [ ]:
import sqlite3
import time

# Function to connect to the SQLite database
def get_db_connection():
    return sqlite3.connect('video_summaries.db')

# Function to create the summaries table if it doesn't exist
def create_summaries_table():
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS summaries (
            video_id TEXT PRIMARY KEY,
            summary TEXT,
            processing_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        ''')
        conn.commit()

# Function to insert or replace a summary into the database with retry mechanism
def insert_or_replace_summary_with_retry(video_id, summary, attempts=3):
    for attempt in range(attempts):
        try:
            conn = sqlite3.connect('video_summaries.db')
            cursor = conn.cursor()
            cursor.execute('''
            INSERT INTO summaries (video_id, summary) VALUES (?, ?)
            ON CONFLICT(video_id) DO UPDATE SET summary=excluded.summary
            ''', (video_id, summary))
            conn.commit()
            print(f"Summary for Video ID: {video_id} saved to the database.")
            return True
        except sqlite3.OperationalError as e:
            print(f"Attempt {attempt+1}: Database operation failed with error: {e}")
            time.sleep(0.5)  # Wait before retrying
        finally:
            conn.close()
    return False

# Function to retrieve and print all summaries from the database
def get_summaries():
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM summaries')
        rows = cursor.fetchall()
        for row in rows:
            print(row)

In [ ]:
import os
import requests
import time
from pprint import pprint

# Assuming you've already defined the database functions from the previous discussion
# insert_or_replace_summary_with_retry(video_id, summary)

def submit_video_for_processing(video_url):
    API_URL = os.getenv("API_URL")
    assert API_URL

    API_KEY = os.getenv("API_KEY")
    assert API_KEY

    TASKS_URL = f"{API_URL}/tasks/external-provider"
    INDEX_ID = '65dbc2fbabd7318d014733be'  # Use your actual INDEX_ID

    headers = {"x-api-key": API_KEY}
    data = {"index_id": INDEX_ID, "url": video_url}

    response = requests.post(TASKS_URL, headers=headers, json=data)
    if response.status_code == 201:
        task_id = response.json().get("_id")
        print(f"Task submitted successfully. Task ID: {task_id}")
        return task_id
    else:
        print(f"Failed to submit task: {response.status_code}")
        pprint(response.json())
        return None

def wait_for_task_completion(task_id):
    API_URL = os.getenv("API_URL")
    headers = {"x-api-key": os.getenv("API_KEY")}
    TASK_STATUS_URL = f"{API_URL}/tasks/{task_id}"

    while True:
        response = requests.get(TASK_STATUS_URL, headers=headers)
        if response.status_code == 200:
            status = response.json().get("status")
            if status == "ready":
                video_id = response.json().get('video_id')
                print(f"Task completed. Video ID: {video_id}")
                return video_id
            elif status == "failed":
                print("Task failed.")
                return None
        else:
            print(f"Failed to get task status: {response.status_code}")
            return None
        time.sleep(10)  # Polling interval


def get_video_summary(video_id):
    API_URL = os.getenv("API_URL")
    assert API_URL

    API_KEY = os.getenv("API_KEY")
    assert API_KEY

    GENERATE_URL = f"{API_URL}/generate"

    headers = {"x-api-key": API_KEY}
    data = {
      "video_id": video_id,
      "prompt": "Make a summary"
    }

    response = requests.post(GENERATE_URL, headers=headers, json=data)
    if response.status_code == 200:
        summary = response.json().get('data')
        print("Video summary generated successfully.")
        return summary
    else:
        print(f"Failed to generate summary: {response.status_code}")
        pprint(response.json())
        return None


def main():
    video_url = input("Enter the video URL: ")
    task_id = submit_video_for_processing(video_url)
    if task_id:
        video_id = wait_for_task_completion(task_id)
        if video_id:
            summary = get_video_summary(video_id)
            if summary:
                # Assuming you have a function to insert or update the summary in the database
                insert_or_replace_summary_with_retry(video_id, summary)
                print("Summary saved to database.")

# Assuming you've already defined the database functions from the previous discussion
def insert_or_replace_summary_with_retry(video_id, summary, attempts=3):
    for attempt in range(attempts):
        try:
            conn = sqlite3.connect('video_summaries.db')
            cursor = conn.cursor()
            # Ensure your SQL syntax matches the capabilities of your SQLite version.
            # The following assumes SQLite 3.24.0 or higher for UPSERT.
            cursor.execute('''
            INSERT INTO summaries (video_id, summary) VALUES (?, ?)
            ON CONFLICT(video_id) DO UPDATE SET summary=excluded.summary;
            ''', (video_id, summary))
            conn.commit()
            print(f"Summary for Video ID: {video_id} saved to the database.")
            return True
        except sqlite3.OperationalError as e:
            print(f"Attempt {attempt + 1}: Database operation failed with error: {e}")
            time.sleep(0.5)  # Wait for 0.5 seconds before retrying
        finally:
            conn.close()
    return False

def get_summaries():
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM summaries')
        rows = cursor.fetchall()
        for row in rows:
            print(row)

if __name__ == "__main__":
    main()

Enter the video URL: https://www.youtube.com/watch?v=FAyKDaXEAgc
Task submitted successfully. Task ID: 65dbef30abd7318d01473468
Task completed. Video ID: 65dbef3748db9fa780cb42bb
Video summary generated successfully.
Summary for Video ID: 65dbef3748db9fa780cb42bb saved to the database.
Summary saved to database.


In [ ]:
def get_summaries():
    with get_db_connection() as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM summaries')
        rows = cursor.fetchall()
        for row in rows:
            print(row)

get_summaries()


('65dbc52a48db9fa780cb4293', 'In the given video, we observe a vlog-style recording featuring two women...', '2024-02-25 23:21:52')
('65dbd0da48db9fa780cb429c', ' Greetings, I\'d be happy to help you understand the contents of the video. This TEDx event video, titled "TEDx MileHigh," can be summarized as follows.\n\nThe video features a speaker, whose name is not explicitly mentioned but can be inferred from the dialogue as Leonardo Silva or Etan Wynnop, giving a talk at the TEDx MileHigh event, which is organized by Chris Anderson. The event is held on a stage with an audience in the background. The video is produced by Chris Anthony and Christy Epps.\n\nThroughout the video, the speaker discusses the importance of catering to the modern audience\'s short attention spans and keeping presentations concise. He shares anecdotes about the decline of long-form content consumption and emphasizes the significance of creating engaging content that can be quickly consumed and shared online. Th